## 决策树
### 一、算法简介
决策树是一种经久不衰的机器学习算法，它采用分治策略（divide-and-conquer）来进行标签预测。

每次迭代选定一种属性作为父节点，根据属性值将样本划分到各子节点中，直到样本分类完全或到达预定的迭代次数算法结束，并返回最后一次子节点预测结果作为模型输出。
（第一次迭代的父节点被特别称作根节点，最后一次迭代的子节点被特别称作叶节点，其它衍生过程中的父子节点被称作内部节点）

_*可见，决策树算法的核心在于找到使每次迭代划分样本尽可能属于同一类别的父节点属性*_


### 二、划分效果评价方法

决策树模型样本划分效果评价方法，至今已经历ID3、C4.5和CART三代发展。

1. 1970年，最先被提出的ID3算法根据最大化信息增益（information gain）来选定迭代的父节点属性。

*但在相同条件下，取值比较多的特征比取值少的特征信息增益大。*

2. 所以，1993年提出C4.5算法，使用增益率（gain rate）来选择最优划分属性，可以减少gain偏好更多取值特征带来的不利影响。

同时，gain rate则会对可取值数目较少的属性有所偏好，因此C4.5算法使用一种启发式特征选择方法：
**首先从候选属性中找出 information gain 高于平均水平的属性，然后在这些属性中找出 gain rate 最高的作为划分属性。**

3. CART与ID3以及C4.5不同，它使用基尼系数（gini coefficien）对属性进行选则，是一个纯粹的二叉树，可用于分类也可用于回归。


In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
import warnings
InteractiveShell.ast_node_interactivity = "all"  # 执行全部行输出命令
warnings.filterwarnings('ignore')

In [ ]:
# 加载数据
from sklearn.datasets import load_boston
data = load_boston()
df = pd.DataFrame(data['data'],columns=data['feature_names'])
df['target']=data['target']
df.info(); df.head()


算法规划：
1. 衡量指标计算函数
2. 数据集分割函数
3. 树生成函数


### 一、指标定义

In [2]:
# 类别概率
from collections import Counter
def ProbE(e,elements):
    return Counter(elements)[e]/len(elements)

# 基尼系数
def gini(elements):
    return 1-np.sum(ProbE(e,elements)**2 for e in set(elements))

# 信息熵
def entropy(elements):
    return -np.sum(ProbE(e,elements)*np.log2(ProbE(e,elements)) for e in set(elements))

# 误差平方和，回归树使用
def mse(elements):
    return np.sum((e-np.mean(elements))**2 for e in elements)

### 二、子集划分

In [42]:
# 二叉树子集划分
def SamplesSplit(df,feature,divide):
    if df[feature].dtype=='object':
        LeftSet = df[df[feature]==divide]
        RightSet= df[~(df[feature]==divide)]
    else:
        LeftSet = df[df[feature]>=divide]
        RightSet= df[~(df[feature]>=divide)]
    return LeftSet,RightSet


### 三、树生成


In [64]:
# 评分
def DivideScore(df,feature,fun,fv):
    """
    默认输入数据集中，label在最后一列
    """
    LeftSet,RightSet = SamplesSplit(df,feature,fv)
    loss = LeftSet.shape[0]/df.shape[0]*fun(LeftSet.iloc[:,-1]) + RightSet.shape[0]/df.shape[0]*fun(RightSet.iloc[:,-1])
    return loss

# 选出特征最佳划分点
def FeatureScore(df,feature,fun):
    """
    Return the best divide and score.
    """
    feature_values = sorted(set(df[feature]),reverse=False)
    return sorted([(divide,DivideScore(df,feature,fun,divide)) for divide in feature_values],key=lambda x:x[1],reverse=False)[0]


# 选出最佳特征
def DivideFeatureSelect(df,fun):
    """
    默认输入数据集中，label在最后一列
    Return the best feature, divide and score.
    """
    features = list(df.columns)[:-1]
    return sorted([(feature,FeatureScore(df,feature,fun)) for feature in features],key=lambda x:x[2],reverse=False)[0]


In [ ]:
temp = DivideFeatureSelect(df,mse)
temp

In [ ]:
def Tree(train,epochs):
    for i in range(epochs):
        for j in range(2**i):
